In [ ]:
import logging
import sys

# Configuração básica do logging
logging.basicConfig(
    level=logging.DEBUG,  # Define o nível mínimo de logs a serem capturados
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',  # Formato das mensagens de log
    handlers=[
        logging.FileHandler("log_analysis.log"),  # Escreve logs em um arquivo
        logging.StreamHandler(sys.stdout)  # Exibe logs no console
    ]
)

# Criação de um logger para este módulo
logger = logging.getLogger(__name__)

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv('./.env')
os.environ["LANGSMITH_PROJECT"] = "Rag Simples"

In [ ]:
import pandas as pd
from langchain.docstore.document import Document

# Criacao de Log (P1)
logger = logging.getLogger(__name__)

try:
    event_log_base = pd.read_csv("datasets/teste.csv")
    logger.info(f"Dataset carregado com sucesso. Número de registros: {len(event_log_base)}.")
except FileNotFoundError as e:
    logger.error(f"Arquivo de CSV nao encontrado: {e}")
    sys.exit(1)
except Exception as e:
    logger.error(f"Erro ao carregar dataset: {e}")
    sys.exit(1)

def create_document(row):
    try:
        content = f"""
        Machine name: {row['MachineName']}.
        Category: {row['Category']}.
        Entry type: {row['EntryType']}.
        Message: {row['Message'].lower()}.
        Source: {row['Source']}.
        Time generated: {row['TimeGenerated']}.
        country: {row['country']},
        regionName: {row['regionName']},
        city: {row['city']}
        """
        # Melhora na estrutura da variavel doc (P1)
        doc = Document(
            page_content=content, 
            metadata={
                "Machine name": row['MachineName'],
                "Category": row['Category'],
                "Entry type": row['EntryType'],
                "Message": row['Message'].lower(),
                "Source": row['Source'],
                "Time generated": row['TimeGenerated'],
                "country": row['country'],
                "regionName": row['regionName'],
                "city": row['city'],
            }
        )
        logger.debug(f"Documento criado para MachineName: {row['MachineName']}, Entry Type: {row['EntryType']}.")
        return doc
    except KeyError as e:
        logger.warning(f"Chave ausente no row: {e}. Row: {row}")
        return None
    except Exception as e:
        logger.error(f"Erro ao criar documento: {e}. Row: {row}")
        return None

logger.info("Criando documentos a partir do dataset.")
docs = [create_document(row) for _, row in event_log_base.iterrows()]

# Remover documentos que não foram criados devido a erros (P1)
# Adicao de loggers necessarios para boa compreensao do pipeline (P1)

docs = [doc for doc in docs if doc is not None]
logger.info(f"Total de documentos criados: {len(docs)}.")

logger.debug(f"Tamanho do conteúdo do primeiro documento: {len(docs[0].page_content)} caracteres.")

In [ ]:
from langchain_ollama.llms import OllamaLLM

# Adicao do logging para a geracao do Llamma (P1)

try:
    logger.info("Configurando o modelo OllamaLLM.")
    llm = OllamaLLM(model="llama3.2", temperature=0) # Alteracao do 1lama 3.1 para o 3.2 (P1)
    logger.info("Modelo OllamaLLM configurado com sucesso.")
except Exception as e:
    logger.error(f"Erro ao configurar o modelo OllamaLLM: {e}")
    sys.exit(1)

In [ ]:
from langchain_text_splitters import SpacyTextSplitter

# Adicao de Logging para o spliting (P1)
try:
    logger.info("Configurando o SpacyTextSplitter.")
    text_splitter = SpacyTextSplitter(chunk_size=1024, chunk_overlap=256, pipeline='pt_core_news_sm')
    logger.info("SpacyTextSplitter configurado com sucesso.")
except Exception as e:
    logger.error(f"Erro ao configurar o SpacyTextSplitter: {e}")
    sys.exit(1)

# Adicao do retorno do total e logging (P1)

logger.info("Dividindo os documentos em chunks.")
all_splits = text_splitter.split_documents(docs)
logger.info(f"Total de chunks criados: {len(all_splits)}.")

In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

# Potencial melhoria: buscar ou treinar um modelo de embeddings pré-treinado para trabalhar com logs

# Adicao do output de logging para a configuracao do huggingface (P1)
try:
    logger.info("Configurando o HuggingFaceEmbeddings.")
    model_name = "sentence-transformers/all-mpnet-base-v2"
    # Possivel melhoria: Rodar em uma GPU da Nvidia usando o pytorch e kuda (P1)
    model_kwargs = {'device': 'cpu'}    
    embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)
    logger.info("HuggingFaceEmbeddings configurado com sucesso.")
except Exception as e:
    logger.error(f"Erro ao configurar HuggingFaceEmbeddings: {e}")
    sys.exit(1)
    
# Adicao do output de logging para configuracao do Chroma (P1)
try:
    logger.info(f"Criando o Chroma vectorstore a partir dos chunks.") 
    vectorstore = Chroma.from_documents(
        documents=all_splits, 
        # Correcao Minoritaria de Codigo (P1)
        embedding=embeddings
    )
except Exception as e:
    logger.error(f"Erro ao configurar o Chroma Vectorstore: {e}")
    sys.exit(1)

In [ ]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 14})

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs):
    try:
        formatted = "\n\n".join(
            f"Metadados: {doc.metadata}\nConteúdo:\n{doc.page_content}" for doc in docs
        )
        # logger.debug("Documentos formatados para o prompt.")
        return formatted
    except Exception as e:
        logger.error(f"Erro ao formatar documentos: {e}")
        return ""

template = """
Utilize as informações abaixo como contexto para responder à pergunta de forma clara, precisa e completa.  
Sempre que possível, cite trechos relevantes das fontes fornecidas para embasar sua resposta.  
Se a resposta não puder ser determinada a partir do contexto, diga apenas que não sabe. Não tente inventar.

### Contexto Disponível:
{context}

### Pergunta:
{question}

### Resposta Útil:"""

try:
    # logger.info("Criando o PromptTemplate para o RAG Chain.")
    custom_rag_prompt = PromptTemplate.from_template(template)
    # logger.info("PromptTemplate criado com sucesso.")
except Exception as e:
    logger.error(f"Erro ao criar PromptTemplate: {e}")
    sys.exit(1)

try:
    # logger.info("Configurando o RAG Chain com EnsembleRetriever.")
    rag_chain = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | custom_rag_prompt
        | llm
        | StrOutputParser()
    )
    # logger.info("RAG Chain configurado com sucesso.")
except Exception as e:
    logger.error(f"Erro ao configurar RAG Chain: {e}")
    sys.exit(1)


In [ ]:
rag_chain.invoke("Qual foi a mensagem registrada pelo sistema no evento gerado em '2020-11-01 06:38:50' pelo 'SecurityCenter'?")

In [ ]:
rag_chain.invoke("Qual foi a mensagem e o horário de término da transação do Windows Installer para o arquivo 'c2rintloc.en-us.16.msi' com id de processo '14504' em 2020-11-13?")

In [ ]:
rag_chain.invoke("Qual é o ID do processo cliente registrado para o término da transação do Windows Installer referente ao arquivo 'c2rint64.16.msi' em '2020-10-26'?")

In [ ]:
rag_chain.invoke("Qual foi o erro identificado no evento com timestamp '2020-11-07 18:21:01', e qual o caminho do arquivo mencionado no registro correspondente?")

In [ ]:
rag_chain.invoke("Qual é a sequência de eventos relacionados ao serviço de Software Protection Platform entre '2020-11-10 16:30:50' e '2020-11-13 18:07:08', incluindo os motivos de re-agendamento e os resultados das verificações de status de licenciamento?")

In [ ]:
rag_chain.invoke("Qual problema foi identificado no evento do 'igfxCUIService2.0.0.0' registrado no timestamp '2020-11-01 06:38:49' com o Event ID '0'?")